In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [23]:
# Load the data
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Assign X and Y from training data
X_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"].values

# Normalize the data for better performance
X_train = X_train / 255.0
test = test / 255.0

# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)

In [38]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.25),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 2, 2, 64)          65600     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 64)              

In [40]:
batch_size = 150
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
252/252 [==============================] - 16s 64ms/step - loss: 0.1238 - accuracy: 0.9630 - val_loss: 0.0596 - val_accuracy: 0.9829
Epoch 2/20
252/252 [==============================] - 16s 63ms/step - loss: 0.0910 - accuracy: 0.9735 - val_loss: 0.0437 - val_accuracy: 0.9869
Epoch 3/20
252/252 [==============================] - 16s 64ms/step - loss: 0.0799 - accuracy: 0.9757 - val_loss: 0.0384 - val_accuracy: 0.9881
Epoch 4/20
252/252 [==============================] - 17s 68ms/step - loss: 0.0659 - accuracy: 0.9808 - val_loss: 0.0357 - val_accuracy: 0.9895
Epoch 5/20
252/252 [==============================] - 17s 67ms/step - loss: 0.0602 - accuracy: 0.9812 - val_loss: 0.0378 - val_accuracy: 0.9886
Epoch 6/20
252/252 [==============================] - 17s 68ms/step - loss: 0.0502 - accuracy: 0.9852 - val_loss: 0.0352 - val_accuracy: 0.9883
Epoch 7/20
252/252 [==============================] - 17s 67ms/step - loss: 0.0469 - accuracy: 0.9858 - val_loss: 0.0327 - val_accuracy:

In [41]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)